In [10]:
import pickle

Load processed data about places

In [11]:
with open('../data/full_data_1680553753.pickle', 'rb') as file:
    data = pickle.load(file)

In [12]:
print(data)

[('ChIJp-rgFA5bFkcRKEAOXj3JZG4', 'Kurant', ['book_store', 'art_gallery', 'cafe', 'point_of_interest', 'food', 'store', 'establishment'], 50.06281010000001, 19.9371409, 'Rynek Główny 36, 31-013 Kraków, Poland', 'http://www.kurant.krakow.pl/', 'https://maps.google.com/?cid=7954704107251318824', None, ['I loved this place. A bookstore full of art items, books, music, vinyls, cd’s, piano, etc.\nA place that you will enjoy to drink a coffee\nand read your book.', 'Im from Italy, so when it comes to food my standard are pretty high. That said, I have to admit that cappuccino and cheesecake were quite good!\nThe place is also charming with nice people and good music all set in beautiful location.', 'The staff is bad, unfriendly, and completely hostile. The guy at the coffee counter gave vague answers when I ordered coffee and cake (he simply asked me to go and sit somewhere and went on with his work like I said nothing; well, there were no seats so he gave me weird looks for walking around).\